In [ ]:
import pydicom
import os
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2 as cv
from scipy import ndimage
from IPython.display import display, HTML
import glob
from multiprocessing import Pool
import concurrent.futures
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
import os
from keras.models import save_model
import glob
from keras.preprocessing import image
from multiprocessing import Pool
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Rectangle
from matplotlib.widgets import RectangleSelector
from collections import Counter
import time
import sys
import io

In [25]:

def dicom_img(num):
    source_path=r"C:\Users\schet\OneDrive\Desktop\MSS-2\Final_dicom_data"
    img_path=os.path.join(source_path,dcm_id['ID_complete'][num])
    dicom_img=pydicom.dcmread(img_path)
    return dicom_img

def transform_HU(dicom_meta):
    image=dicom_meta.pixel_array
    slope=dicom_meta.RescaleSlope
    intercept=dicom_meta.RescaleIntercept
    hu_array=image*slope+intercept
    
    return hu_array

def window_image(image,window_centre,window_width):
    img_min=(window_centre-window_width)/2
    img_max=(window_centre+window_width)/2
    win_image=image.copy()
    win_image[win_image<img_min]=img_min
    win_image[win_image>img_max]=img_max
    
    return win_image


def normalize(image):
    min_=image.min()
    max_=image.max()
    return (image-min_)/(max_-min_)

def map_ct_to_grayscale(ct_image):
    # Normalize pixel values to the range [0, 1]
    normalized_image = cv.normalize(ct_image, None, 0, 1, cv.NORM_MINMAX)
    # Map the normalized values to the 8-bit range [0, 255]
    mapped_image = (normalized_image * 255).astype(np.uint8)

    return mapped_image

def add_padding(image, new_height=512, new_width=512):
    height, width = image.shape[:2]
    final_image = np.zeros((new_height, new_width,3), dtype=np.uint8)

    pad_left = int((new_width - width) // 2)
    pad_top = int((new_height - height) // 2)
    # Replace the pixels with the image's pixels
    final_image[pad_top:pad_top + height, pad_left:pad_left + width] = image
    
    return final_image

def crop(image):
    labeled_blobs, number_of_blobs = ndimage.label(image)
    blob_sizes = np.bincount(labeled_blobs.flatten())

    blob_sizes_sorted = sorted(blob_sizes)
    pos_2large = np.where(blob_sizes == (blob_sizes_sorted[-2]))[0][0]
    blob_0 = labeled_blobs == pos_2large
    blob_0 = np.max(blob_0, axis=-1)
    
    mask=blob_0==0
    rows = np.flatnonzero((~mask).sum(axis=1))
    cols = np.flatnonzero((~mask).sum(axis=0))
    
    # keeping offset from cropping deep
    offset=10
    x_min = max([rows.min() - offset, 0])
    x_max = min([rows.max() + offset + 1, image.shape[0]])
    y_min = max([cols.min() - offset, 0])
    y_max = min([cols.max() + offset + 1, image.shape[1]])
    
    return [x_min,x_max,y_min,y_max]
    

def has_brain(r):

    img = (r.pixel_array * r.RescaleSlope) + r.RescaleIntercept 
    img = img.astype(np.int16) 
  
    brain_img = np.clip(img, 20, 45)
    brain_img[brain_img==20] = 0
    brain_img[brain_img==45] = 0
    status=1 if brain_img.mean()>0.0 else 0

    has_brain = np.sum(brain_only) > 0
    return has_brain

def dcm_array(path):
    meta=pydicom.dcmread(path)
    image=meta.pixel_array
    slope=meta.RescaleSlope
    intercept=meta.RescaleIntercept
    hu_array=image*slope+intercept
    
    return hu_array
# scaling images to gray scale
def scale_dicom_pixel_values(image_hu):
    min_val = np.min(image_hu)
    max_val = np.max(image_hu)
    scaled_image = ((image_hu - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    return scaled_image

def severity(path):
    original_ct=dcm_array(path)
    win_img=window_image(original_ct,35,300)

    h,w=win_img.shape
    x, y = np.meshgrid(np.arange(w), np.arange(h))

    image_flat = win_img.flatten().reshape(-1,1)

    num_clusters=7
    kmeans = KMeans(n_clusters=num_clusters,n_init=10,random_state=42)
    kmeans.fit(image_flat)

    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    # creating the dictonary of information about each cluster
    cluster_info = {'cluster_num': [],'min': [], 'max': [], 'range': [], 'average': [],'pixel_count': []}
    pixel_counts = Counter(labels)
    for i in range(num_clusters):

        cluster_pixels = image_flat[labels == i]

    
        min_val = np.min(cluster_pixels)
        max_val = np.max(cluster_pixels)
        range_val = max_val - min_val
        average_val = np.mean(cluster_pixels)

        # Store information in the dictionary
        cluster_info['cluster_num'].append(i)
        cluster_info['min'].append(min_val)
        cluster_info['max'].append(max_val)
        cluster_info['range'].append(range_val)
        cluster_info['average'].append(average_val)
        cluster_info['pixel_count'].append(pixel_counts[i])
    # ordering the custers based on mean of pixel frequncies of each cluster
    zipped_values = zip(cluster_info['cluster_num'], cluster_info['min'], cluster_info['max'], cluster_info['range'], cluster_info['average'],cluster_info['pixel_count'])
    sorted_values = sorted(zipped_values, key=lambda x: x[4])

    segmented_image = labels.reshape(win_img.shape)
    colored_image = np.zeros_like(win_img)
    for i in range(1):
        colored_image[segmented_image == i] = np.random.randint(0, 255)
    _, binary_mask = cv.threshold(win_img, 150, 255, cv.THRESH_BINARY)

    selected_cluster=sorted_values[-2][0]
    mask = (labels == selected_cluster).reshape(h, w)
    clustred=image_flat[labels == 0]
    background_color = 0
    new_image = np.full_like(win_img, background_color)
    new_image[mask] = win_img[mask]

    # calculating the size of hemohrages relative to the size of brain and presented in percentage.
    hem_cells_den=(sorted_values[-2][-1]/(262144-sorted_values[0][-1]))*100
    #display image
    orignal_scan=scale_dicom_pixel_values(original_ct)
    result_scan=scale_dicom_pixel_values(new_image)
    #enhancing intensity of result image by value 50
    result_scan=cv.add(result_scan,50)
    mask = result_scan > 50
    result_img = np.where(mask, result_scan, orignal_scan)
    
#     plt.figure(figsize=(15,15))
#     plt.subplot(131);plt.imshow(original_ct);plt.axis('off')
#     plt.subplot(132);plt.imshow(new_image); plt.title('Brain & Skull Part');plt.axis('off')
#     plt.subplot(133);plt.imshow(result_img); plt.title(f"Damage Severity= {np.round(hem_cells_den,2)}%");plt.axis('off')
    return [result_img,hem_cells_den]
    

# prediction of models and severity of hemorrhages
# test_path = 'takes jpg image 
def Prediction_Severity_Analysis(models,target_sizes,test_path):
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()

    # calculating the predictions of all models
    predictions = []
    for model, target_size in zip(models, target_sizes):
        img = image.load_img(test_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array/ 255.0
        model_prediction = model.predict(img_array)
        predictions.append(model_prediction)
        
    sys.stdout = old_stdout
    pred=predictions[0][0]
    hemo_status="No Hemorrhages Detected"
    # if prediction is positve then we will peform the severity analysis
    if predictions[0][0]>0.5:
        hemo_status="Hemorrhages Detected"
        file_name = os.path.basename(test_path)
        id_part = file_name.split('_')[1].split('.')[0]
        # change path to extract image to run predictions
        test_dicom_path = fr"C:\Users\schet\OneDrive\Desktop\DICOM_\test_dicom\ID_{id_part}.dcm"
        sev_pred=severity(test_dicom_path)
        return [pred,hemo_status,sev_pred]
    else:
        return [pred,hemo_status]

    
#pre-processing the CT scan and saving in JPG format
def Preprocess(path):
    dcm_meta = pydicom.dcmread(path, force=True)
        
        # Extract pixel array and convert to Hounsfield Units (HU)
    dcm_img = dcm_meta.pixel_array
    image_hu = (dcm_meta.pixel_array * dcm_meta.RescaleSlope) + dcm_meta.RescaleIntercept
        
        # Apply windowing to create different views
    brain_win = window_image(image_hu, 40, 80)         # Brain window
    subdural_win = window_image(image_hu, 80, 200)      # Subdural window
    bone_win = window_image(image_hu, 400, 1800)        # Bone window
        
        # Combine the three window scales into a three-channel image
    image_raw = np.dstack((map_ct_to_grayscale(brain_win), map_ct_to_grayscale(subdural_win), map_ct_to_grayscale(bone_win)))
        
        # Crop the image
    crop_loc = crop(image_raw)
    image_crop = image_raw[crop_loc[0]:crop_loc[1], crop_loc[2]:crop_loc[3]]
        
        # Add padding to the cropped image
    image_pad = add_padding(image_crop)
        
        # Denoise the colored image
    final_image = cv.fastNlMeansDenoisingColored(image_pad, None, 1, 2, 7, 9)
    
    folder=r"C:\Users\schet\OneDrive\Desktop\DICOM_\stored_jpg"
    output_path=os.path.join(folder,os.path.basename(path).replace('dcm','jpg'))
    cv.imwrite(output_path,final_image)
    
    while not os.path.exists(output_path):
        time.sleep(1)

    sev_pred=Prediction_Severity_Analysis(models, target_sizes, output_path)
    return sev_pred


In [23]:
model_paths=[r"C:\Users\schet\OneDrive\Desktop\Model Weights\ResNetrs100.h5",
            r"C:\Users\schet\OneDrive\Desktop\Model Weights\resnet50_model.h5",
            r"C:\Users\schet\OneDrive\Desktop\Model Weights\InceptionV3.h5",
            r"C:\Users\schet\OneDrive\Desktop\Model Weights\VGG19.h5"
           ]

models=[load_model(model_path) for model_path in model_paths]
target_sizes=[(512, 512),(512, 512),(512, 512,3),(512, 512, 3)]
test_path=r"C:\Users\schet\OneDrive\Desktop\DICOM_\test_dicom\ID_00b5b2ae3.dcm"

700

In [22]:
def Preprocesss(path,out_folder):
    dcm_meta = pydicom.dcmread(path, force=True)
        
        # Extract pixel array and convert to Hounsfield Units (HU)
    dcm_img = dcm_meta.pixel_array
    image_hu = (dcm_meta.pixel_array * dcm_meta.RescaleSlope) + dcm_meta.RescaleIntercept
        
        # Apply windowing to create different views
    brain_win = window_image(image_hu, 40, 80)         # Brain window
    subdural_win = window_image(image_hu, 80, 200)      # Subdural window
    bone_win = window_image(image_hu, 400, 1800)        # Bone window
        
        # Combine the three window scales into a three-channel image
    image_raw = np.dstack((map_ct_to_grayscale(brain_win), map_ct_to_grayscale(subdural_win), map_ct_to_grayscale(bone_win)))
        
        # Crop the image
    crop_loc = crop(image_raw)
    image_crop = image_raw[crop_loc[0]:crop_loc[1], crop_loc[2]:crop_loc[3]]
        
        # Add padding to the cropped image
    image_pad = add_padding(image_crop)
        
        # Denoise the colored image
    final_image = cv.fastNlMeansDenoisingColored(image_pad, None, 1, 2, 7, 9)
    
    
    output_path=os.path.join(folder,os.path.basename(path).replace('dcm','jpg'))
    cv.imwrite(output_path,final_image)

In [ ]:
output_folder=r"C:\Users\schet\OneDrive\Desktop\test_1"
test_folder=r"C:\Users\schet\OneDrive\Desktop\test_data_folder"

def preprocess_and_convert(test_folder, output_folder):

    # Iterate through DICOM files in the test folder
    for root, dirs, files in os.walk(test_folder):
        for file in files:
            if file.lower().endswith(".dcm"):
                dicom_path = os.path.join(root, file)
                Preprocesss(dicom_path, output_folder)
preprocess_and_convert(test_folder, output_folder)

In [ ]:


def browse_files():
    # Clear previous outputs
    output_text.delete(1.0, tk.END)

    # Check if there is an existing image label
    if hasattr(process_file, 'img_label'):
        process_file.img_label.destroy()

    file_path = filedialog.askopenfilename(filetypes=[("DICOM files", "*.dcm")])
    if file_path:
        process_button.config(state=tk.NORMAL)
        global selected_file
        selected_file = file_path

def process_file():
    mm = Preprocess(selected_file)

    # Check if there is an existing image label
    if hasattr(process_file, 'img_label'):
        process_file.img_label.destroy()

    if len(mm) > 1:
        output_text.insert(tk.END, f"Status: {mm[1]}\n")
        output_text.insert(tk.END, f"Prediction Probability: {mm[0][0] * 100:.2f}%")

        img = mm[2][0]
        img_pil = Image.fromarray(img)

        # Display the original DICOM image using matplotlib
        dicom_data = pydicom.dcmread(selected_file)
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(dicom_data.pixel_array, cmap='gray')
        plt.axis('off')
        plt.title("Original DICOM Image")

        # Display the processed image using matplotlib
        plt.subplot(1, 2, 2)
        plt.imshow(img_pil)
        plt.axis('off')
        plt.title(f"Damage Severity: {mm[2][1]:.2f}%")

        # Embed the matplotlib plot into the Tkinter window
        canvas = FigureCanvasTkAgg(plt.gcf(), master=output_text.master)
        canvas.draw()

        # Place the embedded plot in the Tkinter window
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        
        # Save the label reference to be able to destroy it later
        process_file.img_label = canvas.get_tk_widget()

    else:
        output_text.insert(tk.END, f"Status: {mm[1]}\n")
        output_text.insert(tk.END, f"Prediction Probability: {mm[0][0] * 100:.2f}%")

# GUI setup
root = tk.Tk()
root.title("DICOM File Processor")

# Set the window size to full-screen
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}")

# Configure the background color
root.configure(bg="lightblue")

# Widgets
browse_button = tk.Button(root, text="Browse", command=browse_files, font=("Times New Roman", 16), height=2, width=20)
process_button = tk.Button(root, text="Run Diagonosis", command=process_file, state=tk.DISABLED, font=("Times New Roman", 16), height=2, width=20)
output_text = tk.Text(root, height=5, width=40, font=("Times New Roman", 14))

# Layout
browse_button.pack(side=tk.LEFT, anchor=tk.NW, padx=10, pady=10)
process_button.pack(side=tk.LEFT, anchor=tk.NW, padx=10, pady=10)
output_text.pack(side=tk.TOP, anchor=tk.NW, padx=10, pady=10)

# Run the GUI
root.mainloop()
